In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import IntSlider, FloatSlider, Button, VBox, Output
from IPython.display import display, clear_output
import time

University facilities data


In [ ]:
facilities = ["Maintenance Hub", "Lecture Hall", "Library", "Dorm", "Labs"]
num_facilities = len(facilities)
coordinates = np.array([[0, 0], [2, 3], [-1, 2], [3, -1], [-2, -2]])  # Approximate coordinates
dist_graph = np.array([
    [0, 300, 200, 400, 250],  # Maintenance Hub
    [300, 0, 250, 350, 400],  # Lecture Hall
    [200, 250, 0, 300, 200],  # Library
    [400, 350, 300, 0, 350],  # Dorm
    [250, 400, 200, 350, 0]   # Labs
])
priorities = np.array([0.0, 0.8, 0.3, 0.5, 0.2])  # Priority for each facility (e.g., Lecture Hall has high priority)

Initialize random seed

In [ ]:
seed = 42
np.random.seed(seed)

Compute fitness considering priorities

In [ ]:
def compute_fitness(solution, dist_graph, priorities, priority_weight):
    total_dist = 0
    priority_factor = 0
    for i in range(len(solution) - 1):
        total_dist += dist_graph[solution[i]][solution[i + 1]]
        if i < len(solution) - 1:  # Exclude the return to start
            priority_factor += priorities[solution[i + 1]]
    adjusted_dist = total_dist / (1 + priority_weight * priority_factor)
    return adjusted_dist  # Lower is better (minimization)


Generate a random route

In [ ]:
def generate_route(num_facilities, start_facility=0):
    route = [start_facility] + list(np.random.permutation(list(range(1, num_facilities))))
    route.append(start_facility)
    return np.array(route)

Apply velocity to a route by performing swaps

In [ ]:
def apply_velocity(route, velocity, num_swaps):
    new_route = route.copy()
    for _ in range(num_swaps):
        idx1, idx2 = np.random.choice(range(1, len(route) - 1), 2, replace=False)  # Exclude start/end
        if velocity > 0:  # Use velocity to decide whether to swap
            new_route[idx1], new_route[idx2] = new_route[idx2], new_route[idx1]
    return new_route


PSO Algorithm

In [ ]:
def run_pso(swarm_size, max_iters, w, c1, c2, priority_weight):
    # Initialize swarm
    swarm = [generate_route(num_facilities) for _ in range(swarm_size)]
    velocities = np.random.uniform(0, 1, swarm_size)  # Scalar velocity for each particle
    pbest = [s.copy() for s in swarm]
    pbest_fitness = [compute_fitness(s, dist_graph, priorities, priority_weight) for s in swarm]

    gbest_idx = np.argmin(pbest_fitness)
    gbest = pbest[gbest_idx].copy()
    gbest_fitness = pbest_fitness[gbest_idx]
    best_length = sum(dist_graph[gbest[i]][gbest[i + 1]] for i in range(len(gbest) - 1))

    # Main loop
    for _ in range(max_iters):
        for i in range(swarm_size):
            # Update velocity (scalar)
            r1, r2 = np.random.random(2)
            velocity = (w * velocities[i] +
                        c1 * r1 * (pbest_fitness[i] - compute_fitness(swarm[i], dist_graph, priorities, priority_weight)) +
                        c2 * r2 * (gbest_fitness - compute_fitness(swarm[i], dist_graph, priorities, priority_weight)))
            velocities[i] = np.clip(velocity, 0, 1)  # Keep velocity in [0, 1]

            # Update position (route) by applying swaps based on velocity
            num_swaps = int(velocities[i] * (num_facilities - 2))  # Max swaps based on velocity
            swarm[i] = apply_velocity(swarm[i], velocities[i], max(1, num_swaps))

            # Ensure route remains valid (start/end fixed, no duplicates)
            inner_part = swarm[i][1:-1]
            if len(set(inner_part)) != len(inner_part):  # Check for duplicates
                swarm[i] = generate_route(num_facilities)

            # Update personal best
            current_fitness = compute_fitness(swarm[i], dist_graph, priorities, priority_weight)
            if current_fitness < pbest_fitness[i]:
                pbest[i] = swarm[i].copy()
                pbest_fitness[i] = current_fitness

            # Update global best
            if current_fitness < gbest_fitness:
                gbest = swarm[i].copy()
                gbest_fitness = current_fitness
                best_length = sum(dist_graph[gbest[i]][gbest[i + 1]] for i in range(len(gbest) - 1))

    return gbest, best_length

Plot function

In [ ]:
def plot_solution(best_solution, best_length):
    plt.clf()
    fig, ax = plt.subplots(figsize=(8, 6))
    ax.scatter(coordinates[:, 0], coordinates[:, 1], c='red', s=100)
    for i, (x, y) in enumerate(coordinates):
        ax.text(x + 0.1, y, facilities[i], fontsize=12)

    for i in range(num_facilities):
        start = coordinates[best_solution[i]]
        end = coordinates[best_solution[i + 1]]
        ax.plot([start[0], end[0]], [start[1], end[1]], 'b-')

    ax.set_title(f"Best Route: {best_length:.2f} meters\nPath: {' -> '.join([facilities[i] for i in best_solution])}")
    ax.set_xlabel("X")
    ax.set_ylabel("Y")
    ax.grid(True)
    plt.show()



GUI Widgets

In [ ]:
swarm_size_slider = IntSlider(min=2, max=50, value=10, description='Swarm Size')
max_iters_slider = IntSlider(min=1, max=100, value=20, description='Max Iterations')
w_slider = FloatSlider(min=0.1, max=1.0, value=0.5, description='Inertia (w)')
c1_slider = FloatSlider(min=0.1, max=2.0, value=1.0, description='Cognitive (c1)')
c2_slider = FloatSlider(min=0.1, max=2.0, value=1.0, description='Social (c2)')
priority_weight_slider = FloatSlider(min=0.0, max=2.0, value=1.0, description='Priority Weight')
run_button = Button(description="Run PSO")

output = Output()

def on_button_clicked(b):
    with output:
        clear_output(wait=True)
        print("Running PSO...")
        start_time = time.time()
        best_solution, best_length = run_pso(
            swarm_size=swarm_size_slider.value,
            max_iters=max_iters_slider.value,
            w=w_slider.value,
            c1=c1_slider.value,
            c2=c2_slider.value,
            priority_weight=priority_weight_slider.value
        )
        print(f"PSO finished in {time.time() - start_time:.2f} seconds.")
        plot_solution(best_solution, best_length)

run_button.on_click(on_button_clicked)



Display GUI

In [ ]:
display(VBox([
    swarm_size_slider, max_iters_slider, w_slider, c1_slider,
    c2_slider, priority_weight_slider, run_button, output
]))